In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import random
import time 
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
import random



In [3]:
def start_crunchbase(Password, Email):
    # Set up Chrome options to open in incognito mode
    chrome_options = Options()
    chrome_options.add_argument("--incognito")
    
    # Set up the Chrome WebDriver
    service = Service(executable_path="C:\\Program Files (x86)\\chromedriver.exe")
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Go to Crunchbase login page
    driver.get("https://www.crunchbase.com/login")
    
    try:
        # Wait for the email field to be visible and find it
        email_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "input[type='email']"))
        )
        email_field.send_keys(Email)

        # Find the password field
        password_field = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "input[type='password']"))
        )
        password_field.send_keys(Password)

        # Submit the form by simulating Enter key press
        password_field.send_keys(Keys.RETURN)
        
        # Wait for login to complete
        time.sleep(5)  # Adjust as needed

    except Exception as e:
        print(f"An error occurred: {e}")

    return driver

# Setup a Crunchbase premium account and provide your email and Password
# driver = start_crunchbase("enter_Password", "enter_email")
driver = start_crunchbase("1loveJacinda#awesome", "benni.musik.234@gmail.com")


# Scraping of Crunchbase companies information including the chrunchbase profile links of the Founders

In [8]:
# Table for Closed Companies-> which is scraped in the code box below
driver.get("https://www.crunchbase.com/discover/organization.companies/efcda900286b4c4f43c5c0381d2a2d18")
# Table for Open Companies-> which is scraped in the code box below
# driver.get("https://www.crunchbase.com/discover/organization.companies/e907c6f88ecb9d4a8ee12b0ed450747f")

In [10]:
Company_Founder_df = pd.DataFrame()
""" 20 Times because Crunchbase allows only to see a window of 
top 1000 companies out of in this case 7230 closed companies. 
To get as much data as possible I sort by different columns so 
always make different companies appear on the top and then in 
the end I will remove the duplicates. I have to make sure to 
sort by noncorrelated and many features to not create a bias. 
Each time that I filter for a different feature I run this 
code again-> in this case filter 15 different times.

For the open companies I just take the top 1000 out of each 
country: Canada, US, UK, Australia
Here I also run the code for each country once
"""
for i in range(20):
    random_sleep(8,18)#manual horizontal and vertical scrolling to make the table load
    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Locate column headers
    column_headers = []
    header_elements = soup.find_all('div', class_='main-content')  # Update selector as needed
    
    for header in header_elements:
        header_text = header.get_text(strip=True)
        column_headers.append(header_text)
    
    # Adjust to ensure the length of column_headers matches the table structure
    column_headers = column_headers[:36]  # Limit to 36 headers if the table has exactly 36 columns
    
    # Scrape each row
    rows = []
    table_rows = soup.find_all('grid-row')  # Update with actual selector
    for row in table_rows:
        row_data = {}
        
        # First Column: Organization Name and Link
        first_col = row.find('grid-cell', {'data-columnid': 'identifier'})
        name = first_col.get_text(strip=True)
        link = first_col.find('a')['href']
        row_data[column_headers[0]] = f"{name};https://www.crunchbase.com{link}"
        
        # Second Column: Number of Contacts
        contacts_col = row.find('grid-cell', {'data-columnid': 'num_contacts'})
        row_data[column_headers[1]] = contacts_col.get_text(strip=True)
        
        # Third Column: Number of Employees
        employees_col = row.find('grid-cell', {'data-columnid': 'num_employees_enum'})
        row_data[column_headers[2]] = employees_col.get_text(strip=True)
        
        # Fourth Column: Number of Founders
        num_founders_col = row.find('grid-cell', {'data-columnid': 'num_founders'})
        row_data[column_headers[3]] = num_founders_col.get_text(strip=True)
        
        # Fifth Column: Founders' Names and Links
        founders_col = row.find('grid-cell', {'data-columnid': 'founder_identifiers'})
        founders = founders_col.find_all('a')
        names = [founder.get_text(strip=True) for founder in founders]
        links = [f"https://www.crunchbase.com{founder['href']}" for founder in founders]
        row_data[column_headers[4]] = f"{', '.join(names)}; {', '.join(links)}"
    
        # Sixth Column: Founded On Date
        founded_on_col = row.find('grid-cell', {'data-columnid': 'founded_on'})
        row_data[column_headers[5]] = founded_on_col.get_text(strip=True)
        
        # Seventh Column: Closed On Date
        closed_on_col = row.find('grid-cell', {'data-columnid': 'closed_on'})
        row_data[column_headers[6]] = closed_on_col.get_text(strip=True)
    
         # Eightth Column: Last Funding Date Column with Link
        last_funding_at_col = row.find('grid-cell', {'data-columnid': 'last_funding_at'})
        row_data[column_headers[7]] = last_funding_at_col.get_text(strip=True)
        
        # Nineth Column: Operating Status
        operating_status_col = row.find('grid-cell', {'data-columnid': 'operating_status'})
        row_data[column_headers[8]] = operating_status_col.get_text(strip=True)
    
        # Tenth Column: Categories Column with multiple links
        categories_col = row.find('grid-cell', {'data-columnid': 'categories'})
        row_data[column_headers[9]] = categories_col.get_text(strip=True)
        
        # Eleventh Column: Short Description
        short_description_col = row.find('grid-cell', {'data-columnid': 'short_description'})
        row_data[column_headers[10]] = short_description_col.get_text(strip=True)
    
         # Twelfth Column: Company Rank Column
        rank_org_company_col = row.find('grid-cell', {'data-columnid': 'rank_org_company'})
        row_data[column_headers[11]] = rank_org_company_col.get_text(strip=True)
        
        # Thirteenth Column: Location
        location_identifiers_col = row.find('grid-cell', {'data-columnid': 'location_identifiers'})
        row_data[column_headers[12]] = location_identifiers_col.get_text(strip=True)
    
        # Fourteenth Column: Investor Type
        investor_type_col = row.find('grid-cell', {'data-columnid': 'investor_type'})
        row_data[column_headers[13]] = investor_type_col.get_text(strip=True)
        
        # Fiveteenth Column: Description
        description_col = row.find('grid-cell', {'data-columnid': 'description'})
        row_data[column_headers[14]] = description_col.get_text(strip=True)
    
         # Sixteenth Column: Investor Stage
        investor_stage_col = row.find('grid-cell', {'data-columnid': 'investor_stage'})
        row_data[column_headers[15]] = investor_stage_col.get_text(strip=True)
        
        # Seventeenth Column: Category Groups
        category_groups_col = row.find('grid-cell', {'data-columnid': 'category_groups'})
        row_data[column_headers[16]] = category_groups_col.get_text(strip=True)
    
        # Eigtteenth Column: Number of Funding Rounds
        num_funding_rounds_col = row.find('grid-cell', {'data-columnid': 'num_funding_rounds'})
        row_data[column_headers[17]] = num_funding_rounds_col.get_text(strip=True)
        
        # Nineteenth Column: Funding Stage
        funding_stage_col = row.find('grid-cell', {'data-columnid': 'funding_stage'})
        row_data[column_headers[18]] = funding_stage_col.get_text(strip=True)
    
         # Twentyth Column: Last Funding Total
        last_funding_total_col = row.find('grid-cell', {'data-columnid': 'last_funding_total'})
        row_data[column_headers[19]] = last_funding_total_col.get_text(strip=True)
        
        # Twentyfirst Column: Last Funding Type
        last_funding_type_col = row.find('grid-cell', {'data-columnid': 'last_funding_type'})
        row_data[column_headers[20]] = last_funding_type_col.get_text(strip=True)
    
        # Twentysecond Column: Last Equity Funding Total
        last_equity_funding_total_col = row.find('grid-cell', {'data-columnid': 'last_equity_funding_total'})
        row_data[column_headers[21]] = last_equity_funding_total_col.get_text(strip=True)
        
        # Twentythird Column: Last Equity Funding Type
        last_equity_funding_type_col = row.find('grid-cell', {'data-columnid': 'last_equity_funding_type'})
        row_data[column_headers[22]] = last_equity_funding_type_col.get_text(strip=True)
    
         # Twentyfourth Column: Equity Funding Total
        equity_funding_total_col = row.find('grid-cell', {'data-columnid': 'equity_funding_total'})
        row_data[column_headers[23]] = equity_funding_total_col.get_text(strip=True)
        
        # Twentyfiveth Column: Funding Total
        funding_total_col = row.find('grid-cell', {'data-columnid': 'funding_total'})
        row_data[column_headers[24]] = funding_total_col.get_text(strip=True)
    
        # Twentysixth Column: Investor Identifiers
        investor_identifiers_col = row.find('grid-cell', {'data-columnid': 'investor_identifiers'})
        row_data[column_headers[25]] = investor_identifiers_col.get_text(strip=True)
        
        # Twentyseventh Column: Number of Lead Investory Column
        num_lead_investors_col = row.find('grid-cell', {'data-columnid': 'num_lead_investors'})
        row_data[column_headers[26]] = num_lead_investors_col.get_text(strip=True)
    
         # Twentyeightth Column:  Number of Investors
        num_investors_col = row.find('grid-cell', {'data-columnid': 'num_investors'})
        row_data[column_headers[27]] = num_investors_col.get_text(strip=True)
        
        # Twentynineth Column: Company Type
        company_type_col = row.find('grid-cell', {'data-columnid': 'company_type'})
        row_data[column_headers[28]] = company_type_col.get_text(strip=True)
    
        # Thirty Column: Website
        website_col = row.find('grid-cell', {'data-columnid': 'website'})
        row_data[column_headers[29]] = website_col.get_text(strip=True)
        
        # Thirtyfirst Column: Twitter
        twitter_col = row.find('grid-cell', {'data-columnid': 'twitter'})
        row_data[column_headers[30]] = twitter_col.get_text(strip=True)
    
         # Thirtysecond Column: Facebook
        facebook_col = row.find('grid-cell', {'data-columnid': 'facebook'})
        row_data[column_headers[31]] = facebook_col.get_text(strip=True)
        
        # Thirtythird Column: LinkedIn
        linkedin_col = row.find('grid-cell', {'data-columnid': 'linkedin'})
        row_data[column_headers[32]] = linkedin_col.get_text(strip=True)
    
        # Thirtyfourth Column: Contact Email
        contact_email_col = row.find('grid-cell', {'data-columnid': 'contact_email'})
        row_data[column_headers[33]] = contact_email_col.get_text(strip=True)
        
        # Thirtyfiveth Column: Number of Articles
        num_articles_col = row.find('grid-cell', {'data-columnid': 'num_articles'})
        row_data[column_headers[34]] = num_articles_col.get_text(strip=True)
    
         # Thirtysixth Column: Hub Tags
        hub_tags_col = row.find('grid-cell', {'data-columnid': 'hub_tags'})
        row_data[column_headers[35]] = hub_tags_col.get_text(strip=True)
        
        # Thirtyseventh Column: Full Description
        description_col = row.find('grid-cell', {'data-columnid': 'description'})
        row_data[column_headers[36]] = description_col.get_text(strip=True)
    
    
        
        rows.append(row_data)
    
    # Convert to DataFrame and save
    df = pd.DataFrame(rows, columns=column_headers[:len(rows[0])])  # Adjust to match row_data keys length
    from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
    

    try:
        next_button = driver.find_element(By.CLASS_NAME, "page-button-next")
        next_button.click()
        print(f"Clicked 'Next' button - Iteration {i+1}")
        # Add a delay if necessary for loading the new page after each click
        time.sleep(2)  # Adjust the sleep time based on load time
    except NoSuchElementException:
        print("Could not find the 'Next' button.")
        break  # Exit loop if "Next" button is not found
    except ElementClickInterceptedException:
        print("The 'Next' button is not clickable at the moment.")
        time.sleep(2)  # Wait before retrying

df.to_csv('crunchbase_data.csv', index=False)

# Close the driver
#driver.quit()

print("Data saved to crunchbase_data.csv")


AttributeError: 'NoneType' object has no attribute 'get_text'

### Create a dataframe with one row for each founder

In [ ]:
# Split the Founders and Links column into separate columns
Company_Founder_df[['Founders', 'Links']] = Company_Founder_df['Founders'].str.split(';', expand=True)

# Strip any extra whitespace from the split values
Company_Founder_df['Links'] = Company_Founder_df['Links'].str.strip()

# Expand the 'Links' column by splitting it on ', ' and exploding it
Company_Founder_df['Links'] = Company_Founder_df['Links'].str.split(',')

# Explode the DataFrame to create one row per link
expanded_df = Company_Founder_df.explode('Links').reset_index(drop=True)

# Drop the original Founders column if it is no longer needed
expanded_df = expanded_df.drop(columns=['Founders'])

# Display the resulting DataFrame
print(expanded_df)


In [ ]:
fifteenth_df = expanded_df

In [ ]:
# Concatenate all DataFrames vertically with a continuous index
combined_df = pd.concat([
    first_df, second_df, third_df, fourth_df, fifth_df,
    sixth_df, seventh_df, eightth_df, nineth_df, tenth_df,
    eleventh_df, twelfth_df, thirteenth_df, fifteenth_df
], ignore_index=True)

# Display the combined DataFrame
print(combined_df)


In [ ]:
# Save the DataFrame to a CSV file
combined_df.to_csv('combined_dataframe.csv', index=False)

print("DataFrame saved as combined_dataframe.csv")


# Scraping of Crunchbase profiles of founders, of which we before scraped the links, to extract Twitter, LinkedIn and further Social Media links

In [ ]:
# Start Crunch base again for Individual Profile scraping
driver = start_crunchbase("1loveJacinda#awesome", "benni.musik.234@gmail.com")

In [ ]:
# Assuming 'expanded_df' is already defined and loaded with URLs in the second column
expanded_df["Links"] = None  # Add an empty column for links

# Loop through each link in expanded_df to scrape links
for index, row in expanded_df.iterrows():
    url = row[1]  # Assuming the second column contains the URLs
    
    # Visit the page
    driver.get(url)
    time.sleep(random.uniform(7, 18))  # Random sleep to mimic human behavior
    
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Find all 'a' tags within 'link-formatter' elements and filter for URLs
    links = [a['href'] for a in soup.select('link-formatter a') if a['href'].startswith('http')]
    
    # Convert links list to a string and add to the DataFrame cell
    expanded_df.at[index, 'Links'] = ', '.join(links)  # Store links as a comma-separated string

# Display the updated DataFrame
expanded_df


In [ ]:
expanded_df.to_csv('your_file_name.csv', index=False) 

In [15]:
Open_startups_Links = pd.read_csv(r"..\open_combined_df.csv")
Open_startups_Links 

,Organization Name,Number of Contacts,Number of Employees,Number of Founders,Founders,Founded Date,Closed Date,Last Funding Date,Operating Status,Industries,...,Number of Investors,Company Type,Website,Twitter,Facebook,LinkedIn,Contact Email,Number of Articles,Hub Tags,Full Description
0,OpenAI;https://www.crunchbase.com/organization...,752,251-500,11,"Andrej Karpathy, Durk Kingma, Elon Musk, Greg ...","Dec 11, 2015",—,"Oct 3, 2024",Active,"Artificial Intelligence (AI),Generative AI,Mac...",...,36,For Profit,www.openai.com,View on Twitter,View on Facebook,View on LinkedIn,support@openai.com,"5,931",Unicorn,OpenAI is an AI research and deployment compan...
1,Perplexity;https://www.crunchbase.com/organiza...,13,11-50,4,"Andy Konwinski, Aravind Srinivas, Denis Yarats...",2022,—,"Oct 15, 2024",Active,"Artificial Intelligence (AI),Chatbot,Generativ...",...,37,For Profit,www.perplexity.ai,View on Twitter,View on Facebook,View on LinkedIn,support@perplexity.ai,278,Unicorn,Perplexity is a search engine platform that us...
2,Anthropic;https://www.crunchbase.com/organizat...,197,501-1000,5,"Daniela Amodei, Dario Amodei, Jack Clark, Sam ...",2021,—,"May 31, 2024",Active,"Artificial Intelligence (AI),Generative AI,Inf...",...,51,For Profit,www.anthropic.com,View on Twitter,View on Facebook,View on LinkedIn,info@anthropic.com,616,Unicorn,Anthropic is an AI-driven research company tha...
3,xAI;https://www.crunchbase.com/organization/xai,1,11-50,5,"Christian Szegedy, Elon Musk, Kyle Kosic, Manu...","Jul 12, 2023",—,"May 26, 2024",Active,"Artificial Intelligence (AI),Generative AI,Inf...",...,18,For Profit,x.ai,View on Twitter,—,View on LinkedIn,—,157,Unicorn,XAI is an AI company that develops a platform ...
4,Polymarket;https://www.crunchbase.com/organiza...,3,11-50,1,Shayne Coplan; https://www.crunchbase.com/pers...,Mar 2020,—,"May 14, 2024",Active,"Blockchain,Cryptocurrency,Predictive Analytics",...,20,For Profit,polymarket.com,View on Twitter,—,View on LinkedIn,—,164,—,Polymarket is a prediction market platform tha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,BuiltGrid;https://www.crunchbase.com/organizat...,3,11-50,2,"John Murphy, Toby Loft; https://www.crunchbase...","Jul 1, 2020",—,"May 1, 2022",Active,"Construction,Information Technology",...,—,For Profit,builtgrid.com/,View on Twitter,View on Facebook,View on LinkedIn,investors@builtgrid.com,—,—,The B2B procurement network connecting custom ...
3996,PickFans;https://www.crunchbase.com/organizati...,1,1-10,2,"Christopher Abbey, Rax Huq; https://www.crunch...","May 1, 2020",—,"Mar 4, 2023",Active,"Communities,Digital Media,News,Sports",...,—,For Profit,www.pickfans.com/,—,—,View on LinkedIn,hello@pickfans.com,—,—,Not just code. We solve big problems through s...
3997,iCandy Interactive Limited;https://www.crunchb...,2,501-1000,1,Kin-Wai Lau; https://www.crunchbase.com/person...,2015,—,"Nov 22, 2021",Active,"Digital Entertainment,Mobile,Web3",...,2,For Profit,www.icandy.io,View on Twitter,View on Facebook,View on LinkedIn,info@icandy.io,19,—,"Award-winning, largest game developer in ANZ a..."
3998,Hub Australia;https://www.crunchbase.com/organ...,41,101-250,1,Brad Krauskopf; https://www.crunchbase.com/per...,"Mar 1, 2011",—,"May 24, 2019",Active,"Commercial Real Estate,Coworking,Real Estate",...,1,For Profit,www.hubaustralia.com,View on Twitter,View on Facebook,View on LinkedIn,hello@hubaustralia.com,17,Pledge 1%,Hub Australia provides workspace solutions to ...


In [ ]:
import pandas as pd
import re

# Define functions to extract different types of links
def extract_linkedin(links):
    # Check if the value is None or NaN
    if pd.isna(links):
        return ''
    return ' '.join([link for link in links.split() if 'linkedin.com' in link])

def extract_facebook(links):
    if pd.isna(links):
        return ''
    return ' '.join([link for link in links.split() if 'facebook.com' in link])

def extract_bing(links):
    if pd.isna(links):
        return ''
    return ' '.join([link for link in links.split() if 'bing.com' in link])

def extract_twitter(links):
    if pd.isna(links):
        return ''
    return ' '.join([link for link in links.split() if 'twitter.com' in link])

def extract_other_links(links):
    if pd.isna(links):
        return ''
    return ' '.join([link for link in links.split() if not any(domain in link for domain in ['linkedin.com', 'facebook.com', 'bing.com', 'twitter.com'])])

# Apply the functions to create new columns with error handling
Open_startups_Links['LinkedIn_Links'] = Open_startups_Links['Links'].apply(lambda x: extract_linkedin(x))
Open_startups_Links['Facebook_Links'] = Open_startups_Links['Links'].apply(lambda x: extract_facebook(x))
Open_startups_Links['Bing_Links'] = Open_startups_Links['Links'].apply(lambda x: extract_bing(x))
Open_startups_Links['Twitter_Links'] = Open_startups_Links['Links'].apply(lambda x: extract_twitter(x))
Open_startups_Links['Other_Links'] = Open_startups_Links['Links'].apply(lambda x: extract_other_links(x))

# Display the modified DataFrame
print(Open_startups_Links)


In [39]:
df = pd.read_csv(r"C:\Users\benni\Downloads\Scraping\Open_startups_Links.csv")

In [41]:
df

,Organization Name,Crunchbase_Links,Links,LinkedIn_Links,Facebook_Links,Bing_Links,Twitter_Links,Other_Links
0,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/andrej-karpa...,"https://karpathy.ai, http://www.twitter.com/ka...",NaN,NaN,NaN,http://www.twitter.com/karpathy,"https://karpathy.ai,"
1,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/durk-kingma,https://www.linkedin.com/in/durk-kingma-58b3564,https://www.linkedin.com/in/durk-kingma-58b3564,NaN,NaN,NaN,NaN
2,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/elon-musk,https://www.twitter.com/elonmusk,NaN,NaN,NaN,https://www.twitter.com/elonmusk,NaN
3,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/greg-brockman,"http://gregbrockman.com, https://www.facebook....","https://www.linkedin.com/in/thegdb,","https://www.facebook.com/gdbgdb,",NaN,http://twitter.com/gdb,"http://gregbrockman.com,"
4,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/ilya-sutskever,"https://www.cs.toronto.edu/~ilya, https://www....","https://www.linkedin.com/in/ilya-sutskever,","https://www.facebook.com/ilya.su,",NaN,NaN,"https://www.cs.toronto.edu/~ilya, https://x.co..."
...,...,...,...,...,...,...,...,...
8476,PickFans;https://www.crunchbase.com/organizati...,https://www.crunchbase.com/person/christopher-...,NaN,NaN,NaN,NaN,NaN,NaN
8477,PickFans;https://www.crunchbase.com/organizati...,https://www.crunchbase.com/person/rax-huq,NaN,NaN,NaN,NaN,NaN,NaN
8478,iCandy Interactive Limited;https://www.crunchb...,https://www.crunchbase.com/person/kin-wai-lau,NaN,NaN,NaN,NaN,NaN,NaN
8479,Hub Australia;https://www.crunchbase.com/organ...,https://www.crunchbase.com/person/brad-krauskopf,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
import pandas as pd
# Remove trailing comma, if present, in the 'Linkedin_url' column
df['LinkedIn_Links'] = df['LinkedIn_Links'].str.rstrip(',')
df['Facebook_Links'] = df['Facebook_Links'].str.rstrip(',')
df['Twitter_Links'] = df['Twitter_Links'].str.rstrip(',')
# Display the updated DataFrame
df


,Organization Name,Crunchbase_Links,Links,LinkedIn_Links,Facebook_Links,Bing_Links,Twitter_Links,Other_Links
0,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/andrej-karpa...,"https://karpathy.ai, http://www.twitter.com/ka...",NaN,NaN,NaN,http://www.twitter.com/karpathy,"https://karpathy.ai,"
1,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/durk-kingma,https://www.linkedin.com/in/durk-kingma-58b3564,https://www.linkedin.com/in/durk-kingma-58b3564,NaN,NaN,NaN,NaN
2,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/elon-musk,https://www.twitter.com/elonmusk,NaN,NaN,NaN,https://www.twitter.com/elonmusk,NaN
3,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/greg-brockman,"http://gregbrockman.com, https://www.facebook....",https://www.linkedin.com/in/thegdb,https://www.facebook.com/gdbgdb,NaN,http://twitter.com/gdb,"http://gregbrockman.com,"
4,OpenAI;https://www.crunchbase.com/organization...,https://www.crunchbase.com/person/ilya-sutskever,"https://www.cs.toronto.edu/~ilya, https://www....",https://www.linkedin.com/in/ilya-sutskever,https://www.facebook.com/ilya.su,NaN,NaN,"https://www.cs.toronto.edu/~ilya, https://x.co..."
...,...,...,...,...,...,...,...,...
8476,PickFans;https://www.crunchbase.com/organizati...,https://www.crunchbase.com/person/christopher-...,NaN,NaN,NaN,NaN,NaN,NaN
8477,PickFans;https://www.crunchbase.com/organizati...,https://www.crunchbase.com/person/rax-huq,NaN,NaN,NaN,NaN,NaN,NaN
8478,iCandy Interactive Limited;https://www.crunchb...,https://www.crunchbase.com/person/kin-wai-lau,NaN,NaN,NaN,NaN,NaN,NaN
8479,Hub Australia;https://www.crunchbase.com/organ...,https://www.crunchbase.com/person/brad-krauskopf,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df.to_csv(r"C:\Users\benni\Downloads\Scraping\Open_startups_Links.csv")